In [2]:
!pip install df-squeezer

In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sn
# Input data files are available in the read-only "../input/" director.ai/competitions/aivkchallengey
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from df_squeezer import df_squeezer

### 0. Imports and requirements

In [4]:
%load_ext autoreload
%autoreload 2

import gc
import os
import sys
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns


import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression


pd.set_option('display.max_columns', None)



os.environ["CUDA_VISIBLE_DEVICES"] = '0'

sys.path.append('../')

### 1. Reading Data

In [5]:

train = pd.read_parquet("train.parquet")
test = pd.read_parquet("test.parquet")


In [6]:
train.head()

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,flag
0,1678548,3,8,7,17,16,9,1,9,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,3,2,1,0,0,0
1,2834188,11,3,2,0,7,14,8,2,5,1,0,2,13,6,16,5,4,8,1,0,1,1,1,1,2,17,0,1,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,2,4,1,0,0,0
2,811902,11,9,6,11,13,14,8,2,5,1,0,2,11,6,16,5,4,8,1,1,1,1,1,11,2,17,0,1,1,0,0,0,0,0,0,0,0,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,0,0,0
3,836450,1,16,0,13,0,4,9,5,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,0
4,1769024,15,9,9,4,8,1,11,1,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,1,1,0


### 2. Memory Reduction

In [7]:
df_squeezer(train, edit=True, report=False)

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,flag
0,1678548,3,8,7,17,16,9,1,9,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,3,2,1,0,0,0
1,2834188,11,3,2,0,7,14,8,2,5,1,0,2,13,6,16,5,4,8,1,0,1,1,1,1,2,17,0,1,1,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,2,4,1,0,0,0
2,811902,11,9,6,11,13,14,8,2,5,1,0,2,11,6,16,5,4,8,1,1,1,1,1,11,2,17,0,1,1,0,0,0,0,0,0,0,0,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,0,0,0
3,836450,1,16,0,13,0,4,9,5,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,0
4,1769024,15,9,9,4,8,1,11,1,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17659828,2103608,4,1,9,4,8,1,11,10,2,4,0,2,2,6,16,5,4,8,1,1,1,1,1,10,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,1,1,0
17659829,2169551,7,15,4,12,10,6,13,15,2,3,0,2,2,6,16,5,4,8,1,0,1,1,1,16,2,17,1,1,1,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,1,3,2,1,0,0,0
17659830,1613436,1,8,9,16,5,9,9,10,3,3,0,2,10,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,0
17659831,1604231,6,17,13,12,1,2,4,2,2,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,3,0,1,0,0,0


In [8]:
df_squeezer(test, edit=True, report=False)

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,id_x_rn
0,1472943,2,10,4,16,16,13,10,10,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,1472943_x_2
1,660465,5,6,14,9,7,3,11,0,2,3,0,2,4,6,16,5,4,8,1,0,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,1,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,660465_x_5
2,1788193,3,1,0,9,9,7,2,16,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,1788193_x_3
3,2767146,3,15,8,1,16,6,13,14,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,9,5,4,0,0,0,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,3,1,0,0,2767146_x_3
4,2601698,9,19,10,4,8,1,11,4,2,3,0,2,2,6,16,5,4,8,1,1,1,1,1,9,5,4,0,0,0,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,1,1,2601698_x_9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654063,2159273,7,9,1,15,9,1,11,6,2,3,0,2,0,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,2159273_x_7
654064,320081,7,0,4,3,5,6,13,13,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,320081_x_7
654065,2663677,4,15,4,1,15,13,13,7,5,3,0,2,2,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,3,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0,2663677_x_4
654066,1382804,18,11,16,11,13,14,8,14,0,5,0,2,1,6,16,5,4,8,1,0,1,1,1,1,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,0,3,4,1,2,4,1,0,0,1382804_x_18


### 3. Model Training

In [9]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(columns=["id", "rn", "flag"]), train.flag, stratify=train.flag, test_size=0.3)
X_test = test.drop(columns=["id", "rn", "id_x_rn"])
del train, test
_= gc.collect()

In [10]:
y_train.value_counts()

flag
0    11946173
1      415710
Name: count, dtype: int64

In [11]:
(1/y_val.value_counts(normalize=True)).to_dict()

{0: 1.0347985043914896, 1: 29.736867215608356}

In [12]:
lr = LogisticRegression(class_weight=(1/y_val.value_counts(normalize=True)).to_dict(), max_iter=10000)
lr.fit(X_train, y_train)
train_preds = lr.predict(X_train)
val_preds = lr.predict(X_val)
test_preds = lr.predict(X_test)

In [13]:
train_auc = roc_auc_score(y_train, train_preds)
val_auc = roc_auc_score(y_val, val_preds)
print(f"Train AUC: {train_auc}\n Val AUC: {val_auc}")

Train AUC: 0.5738594023924244
 Val AUC: 0.5738221902840167


In [14]:
ss = pd.DataFrame({
    "id": pd.read_parquet("test.parquet", columns=["id_x_rn"])["id_x_rn"],
    "target": test_preds
})

In [15]:
ss.target.value_counts()

target
0    397785
1    256283
Name: count, dtype: int64

In [16]:
ss.to_parquet("submission_logistics.parquet", index=False)